In [ ]:
from canns.data.loaders import load_grid_data

grid_data = load_grid_data()

print(f"Keys: {list(grid_data.keys())}")
print(f"t: {grid_data['t'].shape}, x: {grid_data['x'].shape}, y: {grid_data['y'].shape}")
print(f"spike type: {type(grid_data['spike'])}")


In [ ]:
from canns.analyzer import data

spike_cfg = data.SpikeEmbeddingConfig(
    smooth=True,
    speed_filter=False,
    min_speed=2.5,
)
spikes, xx, yy, tt = data.embed_spike_trains(grid_data, config=spike_cfg)

print(f"Embedded spikes: {spikes.shape}")


In [ ]:
from canns.analyzer.visualization import PlotConfigs

tda_cfg = data.TDAConfig(maxdim=1, do_shuffle=False, show=False, progress_bar=True)
persistence = data.tda_vis(embed_data=spikes, config=tda_cfg)

grid_data_embedded = dict(grid_data)
grid_data_embedded["spike"] = spikes

decoding = data.decode_circular_coordinates_multi(
    persistence_result=persistence,
    spike_data=grid_data_embedded,
    num_circ=2,
)

config = PlotConfigs.cohomap(show=False)

data.plot_cohomap_multi(
    decoding_result=decoding,
    position_data={"x": grid_data["x"], "y": grid_data["y"]},
    config=config,
)


In [ ]:
import numpy as np
from canns.analyzer import data
from canns.analyzer.visualization import PlotConfigs

coords = np.asarray(decoding.get("coords"))

_t = np.asarray(grid_data["t"]).ravel()
_x = np.asarray(grid_data["x"]).ravel()
_y = np.asarray(grid_data["y"]).ravel()

t_use, x_use, y_use, coords_use, _ = data.align_coords_to_position(
    t_full=_t,
    x_full=_x,
    y_full=_y,
    coords2=coords,
    use_box=True,
    times_box=decoding.get("times_box", None),
    interp_to_full=True,
)

coords_use = data.apply_angle_scale(coords_use, "rad")

plot_config = PlotConfigs.path_compare(show=False)

data.plot_path_compare(x_use, y_use, coords_use, config=plot_config)


In [ ]:
import numpy as np
from canns.analyzer import data
from canns.analyzer.visualization import PlotConfigs

coords = np.asarray(decoding.get("coords"))
coordsbox = np.asarray(decoding.get("coordsbox"))

traj_cfg = PlotConfigs.cohospace_trajectory(show=False)

data.plot_cohospace_trajectory(
    coords=coords[:, :2],
    times=None,
    subsample=2,
    config=traj_cfg,
)

neuron_cfg = PlotConfigs.cohospace_neuron(show=False)

data.plot_cohospace_neuron(
    coords=coordsbox[:, :2],
    activity=spikes,
    neuron_id=130,
    mode="fr",
    top_percent=1,
    config=neuron_cfg,
)


In [ ]:
from pathlib import Path
from canns.analyzer.visualization import PlotConfigs

out_dir = Path("Results/asa")
out_dir.mkdir(parents=True, exist_ok=True)

heatmap = data.compute_fr_heatmap_matrix(spikes, transpose=True)
heatmap_cfg = PlotConfigs.fr_heatmap(show=False, save_path=str(out_dir / "fr_heatmap.png"))

data.save_fr_heatmap_png(heatmap, config=heatmap_cfg)

frm_res = data.compute_frm(
    spikes,
    grid_data["x"],
    grid_data["y"],
    neuron_id=130,
    bins=50,
    min_occupancy=1,
    smoothing=True,
    sigma=1.0,
    nan_for_empty=True,
)

frm_cfg = PlotConfigs.frm(show=False, save_path=str(out_dir / "frm.png"))

data.plot_frm(frm_res.frm, config=frm_cfg, dpi=200)
